In [ ]:
Import Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlt

data = pd.read_stata("callreports_final.dta", chunksize=100000, preserve_dtypes=True)

df_old = pd.DataFrame()

for chunk in data:
    df_old=df_old.append(chunk)

In [ ]:
df_2014_2019 = pd.read_csv(r"C:\Users\AlexanderSchlechter\Google Drive\Bachelorarbeit\16.01.2020\BachelorThesisASch\AdditionalData\df_2014-2019", 
                           index_col=0)

#concat two imported dataframes
df_raw = pd.concat([df_2014_2019, df_old])

In [ ]:
Additional Settings

In [ ]:
%matplotlib inline
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from matplotlib import ticker
import matplotlib.patches as mpatches

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 18

#setting matplotlib style
plt.style.use('default')

#setting params for matplolib plots
plt.rcParams['font.size']=BIGGER_SIZE          # controls default text sizes
plt.rcParams['axes.titlesize']=BIGGER_SIZE     # fontsize of the axes title
plt.rcParams['axes.labelsize']=BIGGER_SIZE    # fontsize of the x and y labels
plt.rcParams['xtick.labelsize']=BIGGER_SIZE    # fontsize of the tick labels
plt.rcParams['ytick.labelsize']=BIGGER_SIZE    # fontsize of the tick labels
plt.rcParams['legend.fontsize']=BIGGER_SIZE    # legend fontsize
plt.rcParams['figure.titlesize']=BIGGER_SIZE  # fontsize of the figure title
plt.rcParams['xtick.bottom']=True  # fontsize of the figure title
plt.rcParams['xtick.major.bottom']=True  # fontsize of the figure title
plt.rcParams['xtick.major.size']=3.5  # fontsize of the figure title
#plt.rcParams['xtick.major.width']=1  # fontsize of the figure title
plt.rcParams['xtick.major.top']=False  # fontsize of the figure title
plt.rcParams['xtick.minor.bottom']=True  # fontsize of the figure title
plt.rcParams['xtick.minor.size']=3  # fontsize of the figure title
#plt.rcParams['legend.fontsize'] = 22

#setting grid style
plt.rcParams['grid.color'] = 'k'
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.linewidth'] = 0.5




Structural adjustments on dataset

In [ ]:
#removing investment banks Goldman Sachs and Morgan Stanley
df_raw = df_raw[(df_raw.rssdid!=1456501) & (df_raw.bhcid!=2380443)]

Timeseries labels for xaxis 

In [ ]:
years = [str(year) for year in range(1976, 2020)]

years16 = [str(year) for year in range(1976, 2014, 4)]

In [ ]:
def plot_crisis(ax):
    #1980 crisis
    ax.axvspan(16, 19, alpha=0.5, color='red')


    #1981Q3-1982Q4 crisis
    ax.axvspan(22, 27, alpha=0.5, color='red')

    #1990Q3-1991Q2 crisis
    ax.axvspan(58, 61, alpha=0.5, color='red')

    #2001Q2-2001Q4 crisis
    ax.axvspan(101, 103, alpha=0.5, color='red')

    #2007Q4-2009Q3 crisis
    ax.axvspan(127, 134, alpha=0.5, color='red')

Analysing asset side & liability side

In [ ]:
#getting relevant data
df_assets = df_raw[["rssdid","bhcid",  "name", 'year', "quarter","assets", "cash"
             ,"fedfundsrepoasset" ,"securities","loansnet", "tradingassets"]]

df_liab = df_raw[["rssdid", "name", 'year', "quarter","assets", "equity","fedfundsrepoliab" ,
             "deposits","foreigndep", "otherborrowedmoney", "tradingliabilities","subordinateddebt", "liabilities"]]

#aggregate data by year and quarter
df_agg = df_assets.groupby(["year", "quarter"]).sum()
df_agg_liab = df_liab.groupby(["year", "quarter"]).sum()

#reset index
df_agg =df_agg.reset_index()
df_agg_liab =df_agg_liab.reset_index()

#clean fedfundsrepoasset by putting average for null value
average = (df_agg.loc[103].fedfundsrepoasset + df_agg.loc[105].fedfundsrepoasset) / 2
df_agg.loc[104].fedfundsrepoasset = average

#computing other assets and other liabilities
df_agg["otherassets"] = df_agg["assets"] - (df_agg.iloc[:,5:10].sum(axis=1))
df_agg["otherliab"] = df_agg_liab["assets"] - (df_agg_liab.iloc[:,4:11].sum(axis=1))
df_agg_liab["otherliab"] = df_agg_liab["assets"] - (df_agg_liab.iloc[:,4:11].sum(axis=1))

#clean dataframe of nan, inf and zeros


#getting right yscale
df_agg_scaled = df_agg/1000000000

#remove liabilities, its unnessesary
df_agg_liab.drop(["liabilities"], axis=1 , inplace=True)

#log liabilities
df_agg_liab_log = np.log(df_agg_liab.iloc[:,3:])
df_agg_liab_log = pd.concat([df_agg_liab.iloc[:,:3],df_agg_liab_log], axis=1)
df_agg_liab

In [ ]:
fig = plt.figure(figsize=(20,15))

#ax_twin = ax[1].twinx()

ax = fig.add_subplot(311)

#graph: absolute assets
ax.set_ylabel("Trillions Dollar")
df_agg_scaled.plot.area(ax=ax, y=["cash","fedfundsrepoasset" ,"securities",
                           "loansnet", "tradingassets", "otherassets"])
ax.set_xticks(range(0,176,4))
ax.xaxis.set_minor_locator(AutoMinorLocator(4))
ax.set_xticklabels(years, rotation=60)
#ax.set_xlim(0, 151)
ax.legend()
ax.grid()

#ax.yaxis.set_major_formatter(mlt.ticker.StrMethodFormatter("{x:g}")) 

#graph: log(assets)
df_agg_log = np.log(df_agg_scaled)
ax1 = fig.add_subplot(312)
ax1.set_xticks(range(0,176,4))
ax1.xaxis.set_minor_locator(AutoMinorLocator(4))
df_agg_log.assets.plot(ax=ax1, color="C0")
ax1.set_xticklabels(years, rotation=60)
ax1.set_ylabel("log(assets)")
ax1.legend(["log(assets)"])
ax1.grid()
#ax1.set_xlim(0, 151)



#graph: cyclical of log(assets)
ax3 = fig.add_subplot(313)
cycle, trend = sm.tsa.filters.hpfilter(df_agg_log.assets, 1600)
df_agg_log["cycle"] = cycle
df_agg_log.cycle.plot(ax=ax3)

ax3.set_ylabel("log(assets)")
ax3.axhline(color="black")
ax3.set_xticks(range(0,176,4))
ax3.xaxis.set_minor_locator(AutoMinorLocator(4))
ax3.set_xticklabels(years, rotation=60)
red_patch = mpatches.Patch(color='red',alpha=0.5)
red_patch1 = mpatches.Patch(color='C0', lw=0.2)
ax3.legend(handles=[red_patch, red_patch1], labels=["crisis", "detrended log(assets)"])


#ax3.legend()
#ax3.legend(["cycle:log(assets)", "crisis", "crisis"])
ax3.grid()

'''
#1980 crisis
ax3.axvspan(16, 19, alpha=0.5, color='red')
#ax3.annotate("crisis", xy=(16,0.05), xytext=(8,0.06), 
#arrowprops=dict(facecolor='black', shrink=0.05))


#1981Q3-1982Q4 crisis
ax3.axvspan(22, 27, alpha=0.5, color='red')

#1990Q3-1991Q2 crisis
ax3.axvspan(58, 61, alpha=0.5, color='red')

#2001Q2-2001Q4 crisis
ax3.axvspan(101, 103, alpha=0.5, color='red')

#2007Q4-2009Q3 crisis
ax3.axvspan(127, 134, alpha=0.5, color='red')
'''
plot_crisis(ax3)

#plot twin axis for second graph
ax_twin = ax1.twinx()
ax_twin.plot(df_agg_scaled.assets, "y")

ax_twin.legend(["assets"], loc=[0.008, 0.72])
ax_twin.set_ylabel("Trillions Dollar")


plt.tight_layout()

plt.savefig('LatexVorlage/graphs/DescriptiveStats/OtherAnalysis_clean_AssetDistribution_7613.png')